In [1]:
# Lstm
import pandas as pd
import numpy as np
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
import random


c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import unknown_detection_training, unknown_detection_validation, unknown_detection_full

In [3]:
from models import Lstm

In [4]:
results = pd.read_pickle('results\\model_lstm_final_version.pkl')

In [6]:
res = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max'])

for lstm_unit in [64,128]:
    for epoch in [20, 30]:
        for lr in [0.001, 0.01, 0.1]:
            for batch in [32,64,128]:
                model = Lstm(lstm_units=lstm_unit, dropout_rate=0.1, learning_rate=lr, num_classes=2, batch_size=batch, epoch=epoch, model_path='models/lstm_unknown.h5')
                model.train(unknown_detection_training, unknown_detection_validation)
                res = np.concatenate((res, pd.DataFrame([[lstm_unit,0.1, lr, epoch, batch, model.history.history['loss'][-1], model.history.history['accuracy'][-1], model.history.history['val_loss'][-1], model.history.history['val_accuracy'][-1]]], 
                                                                        columns=['lstm_units', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max'])))

Epoch 1/20
214/214 [==============================] - ETA: 0s - loss: 0.7079 - accuracy: 0.5936
Epoch 1: val_accuracy improved from -inf to 0.65681, saving model to models\lstm_unknown.h5
214/214 [==============================] - 48s 144ms/step - loss: 0.7079 - accuracy: 0.5936 - val_loss: 0.6191 - val_accuracy: 0.6568
Epoch 2/20
214/214 [==============================] - ETA: 0s - loss: 0.5837 - accuracy: 0.6976
Epoch 2: val_accuracy improved from 0.65681 to 0.69565, saving model to models\lstm_unknown.h5
214/214 [==============================] - 22s 104ms/step - loss: 0.5837 - accuracy: 0.6976 - val_loss: 0.5870 - val_accuracy: 0.6956
Epoch 3/20
214/214 [==============================] - ETA: 0s - loss: 0.4964 - accuracy: 0.7625
Epoch 3: val_accuracy improved from 0.69565 to 0.70300, saving model to models\lstm_unknown.h5
214/214 [==============================] - 23s 109ms/step - loss: 0.4964 - accuracy: 0.7625 - val_loss: 0.5710 - val_accuracy: 0.7030
Epoch 4/20
214/214 [========

In [9]:
#pd.DataFrame(res,columns=['num_heads', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'num_layer', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max']).to_pickle('transformer_wyniki_.pkl')
pd.DataFrame(res,columns=['lstm_units', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max']).to_pickle('results/lstm_wyniki_known_pred.pkl')

In [5]:
results.columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max']


In [6]:
results[results['val_accuracy'] == results['val_accuracy'].max()]

,lstm_units,dropout_rate,epoch,batch,loss,loss_max,accuracy,accuracy_max,val_loss,val_loss_max,val_accuracy,val_accuracy_max
4,64,0.2,20,64,0.434148,"[2.4646902084350586, 1.4764158725738525, 1.160...",0.870311,"[0.3031783699989319, 0.5757989287376404, 0.666...",0.736218,"[1.6360548734664917, 1.2466334104537964, 1.031...",0.816564,"[0.5272138714790344, 0.6362165212631226, 0.707..."


In [12]:
model = Lstm(64, 0.2, 20, 32, 0.001, (39,44), 30, 'models\\lstm_mod.h5')

In [13]:
model.train(label_detection_validation)

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 2.7642 - accuracy: 0.3926
Epoch 1: accuracy improved from -inf to 0.39262, saving model to models\lstm_mod.h5
213/213 [==============================] - 17s 46ms/step - loss: 2.7642 - accuracy: 0.3926
Epoch 2/20
213/213 [==============================] - ETA: 0s - loss: 1.5931 - accuracy: 0.6222
Epoch 2: accuracy improved from 0.39262 to 0.62224, saving model to models\lstm_mod.h5
213/213 [==============================] - 10s 44ms/step - loss: 1.5931 - accuracy: 0.6222
Epoch 3/20
212/213 [============================>.] - ETA: 0s - loss: 1.2429 - accuracy: 0.6555
Epoch 3: accuracy improved from 0.62224 to 0.65593, saving model to models\lstm_mod.h5
213/213 [==============================] - 9s 44ms/step - loss: 1.2420 - accuracy: 0.6559
Epoch 4/20
213/213 [==============================] - ETA: 0s - loss: 1.0566 - accuracy: 0.6890
Epoch 4: accuracy improved from 0.65593 to 0.68903, saving model to models\lstm_mod.h5

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
from dataset import label_detection_training, label_detection_validation, silence_detection_training, silence_detection_validation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import TensorflowDataset

In [3]:
train = pd.read_pickle('extracted_features\\features_training.pkl')
y_train = np.array([x[1] for x in train])
labels = list(np.unique(y_train))
train = TensorflowDataset('extracted_features\\features_training.pkl', labels=labels).dataset
train = train.shuffle(len(train), reshuffle_each_iteration=True)
val = TensorflowDataset('extracted_features\\features_validation.pkl', labels=labels).dataset
val = val.shuffle(len(val), reshuffle_each_iteration=True)

In [8]:
results = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])


In [9]:
len(pd.read_pickle('extracted_features\\features_training.pkl'))

57923

In [15]:
# Set your model's hyperparameters
for lstm_units in [64, 128]:
    for dropout_rate in [0.2, 0.4]:
            for epoch in [10, 20]:
                  for batch_size in [32, 64]:
                        print(f'Running model with {lstm_units} units, {dropout_rate} dropout rate, {epoch} epochs and {batch_size} batch size')

                        num_classes = len(np.unique(labels))  # Number of unique classes in your dataset

                        input_shape = (39, 44)

                        model = Sequential([
                            Bidirectional(LSTM(lstm_units, return_sequences=True), input_shape=input_shape),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Bidirectional(LSTM(lstm_units, return_sequences=True)),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units, return_sequences=True),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(64, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(32, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(num_classes, activation='softmax')
                        ])


                        # Compile the model
                        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                        # Train the model with your training set and validate it with your validation set
                        history = model.fit(train.batch(batch_size), epochs=epoch, validation_data=val.batch(batch_size))

                        results = np.concatenate((results, pd.DataFrame([[lstm_units, dropout_rate, epoch, batch_size, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                        columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)


Running model with 64 units, 0.2 dropout rate, 10 epochs and 32 batch size
Epoch 1/10
1811/1811 [==============================] - 218s 109ms/step - loss: 2.4271 - accuracy: 0.3132 - val_loss: 1.5653 - val_accuracy: 0.5462
Epoch 2/10
1811/1811 [==============================] - 159s 87ms/step - loss: 1.4517 - accuracy: 0.5814 - val_loss: 1.1923 - val_accuracy: 0.6553
Epoch 3/10
1811/1811 [==============================] - 167s 92ms/step - loss: 1.1520 - accuracy: 0.6685 - val_loss: 1.0082 - val_accuracy: 0.7170
Epoch 4/10
1811/1811 [==============================] - 183s 101ms/step - loss: 1.0003 - accuracy: 0.7115 - val_loss: 0.9068 - val_accuracy: 0.7343
Epoch 5/10
1811/1811 [==============================] - 180s 99ms/step - loss: 0.8990 - accuracy: 0.7417 - val_loss: 0.9288 - val_accuracy: 0.7396
Epoch 6/10
1811/1811 [==============================] - 182s 100ms/step - loss: 0.8240 - accuracy: 0.7619 - val_loss: 0.8555 - val_accuracy: 0.7611
Epoch 7/10
1811/1811 [==================

KeyboardInterrupt: 

In [16]:
results_f = pd.DataFrame(results)

In [17]:
results_f

,0,1,2,3,4,5,6,7,8,9,10,11
0,64,0.2,1,32,2.370784,[2.370784044265747],0.333063,[0.3330628573894501],1.522492,[1.5224920511245728],0.5584,[0.5583995580673218]
1,64,0.2,10,32,0.660897,"[2.427072286605835, 1.4517465829849243, 1.1519...",0.807175,"[0.31317439675331116, 0.581358015537262, 0.668...",0.76055,"[1.5653058290481567, 1.1923205852508545, 1.008...",0.795528,"[0.546190083026886, 0.655339777469635, 0.71697..."
2,64,0.2,10,64,0.600205,"[2.488501787185669, 1.461273193359375, 1.12221...",0.823559,"[0.30050238966941833, 0.5798560380935669, 0.67...",0.765887,"[1.6190937757492065, 1.1751519441604614, 1.014...",0.790527,"[0.5310385227203369, 0.6641659140586853, 0.712..."
3,64,0.2,20,32,0.459687,"[2.3511674404144287, 1.395878791809082, 1.1104...",0.865408,"[0.3353935480117798, 0.5998998880386353, 0.681...",0.708278,"[1.5098788738250732, 1.139453411102295, 1.0019...",0.81627,"[0.559723436832428, 0.6802000403404236, 0.7131..."
4,64,0.2,20,64,0.434148,"[2.4646902084350586, 1.4764158725738525, 1.160...",0.870311,"[0.3031783699989319, 0.5757989287376404, 0.666...",0.736218,"[1.6360548734664917, 1.2466334104537964, 1.031...",0.816564,"[0.5272138714790344, 0.6362165212631226, 0.707..."
5,64,0.4,10,32,0.922474,"[2.8702659606933594, 1.9048563241958618, 1.527...",0.74233,"[0.19589799642562866, 0.4504428207874298, 0.56...",0.943816,"[2.0041897296905518, 1.4960237741470337, 1.257...",0.755075,"[0.41791704297065735, 0.5720800161361694, 0.64..."
6,64,0.4,10,64,0.862444,"[3.029785394668579, 1.9737733602523804, 1.5499...",0.757747,"[0.16844776272773743, 0.4341798722743988, 0.55...",0.950969,"[2.0705583095550537, 1.57509183883667, 1.30928...",0.755663,"[0.38849660754203796, 0.5460429787635803, 0.62..."
7,64,0.4,20,32,0.695517,"[2.872177839279175, 1.8928440809249878, 1.5408...",0.806105,"[0.19755537807941437, 0.45203113555908203, 0.5...",0.809129,"[1.9758355617523193, 1.4280952215194702, 1.273...",0.794793,"[0.42586055397987366, 0.5837010741233826, 0.64..."
8,64,0.4,20,64,0.647044,"[3.02612042427063, 1.9941836595535278, 1.55550...",0.815583,"[0.1640971601009369, 0.42404571175575256, 0.55...",0.829114,"[2.168159008026123, 1.5454410314559937, 1.2715...",0.788026,"[0.3620182275772095, 0.5489850044250488, 0.644..."


In [18]:
results_f.to_pickle('results\\model_lstm_final_version.pkl')

In [97]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization

results = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

# Set your model's hyperparameters
for lstm_units in [64, 128, 256]:
    for dropout_rate in [0.2, 0.4, 0.6]:
            for epoch in [10, 20, 30, 40]:
                  for batch in [32, 64, 128]:

                        num_classes = len(y_train_encoded)  # Number of unique classes in your dataset

                        input_shape = (39, 44)

                        model = Sequential([
                            Bidirectional(LSTM(lstm_units, return_sequences=True), input_shape=input_shape),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Bidirectional(LSTM(lstm_units, return_sequences=True)),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units, return_sequences=True),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(64, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(32, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(num_classes, activation='softmax')
                        ])


                        # Compile the model
                        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                        # Train the model with your training set and validate it with your validation set
                        epochs = epoch
                        batch_size = batch
                        history = model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_encoded))

                        results = np.concatenate((results, pd.DataFrame([[lstm_units, dropout_rate, epoch, batch, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                        columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)


Epoch 1/10
214/214 [==============================] - 24s 79ms/step - loss: 7.0679 - accuracy: 0.0974 - val_loss: 4.0350 - val_accuracy: 0.1478
Epoch 2/10
214/214 [==============================] - 16s 74ms/step - loss: 2.9956 - accuracy: 0.2135 - val_loss: 2.5939 - val_accuracy: 0.2307
Epoch 3/10
214/214 [==============================] - 16s 75ms/step - loss: 2.3891 - accuracy: 0.2976 - val_loss: 2.3668 - val_accuracy: 0.3070
Epoch 4/10
214/214 [==============================] - 16s 77ms/step - loss: 2.1566 - accuracy: 0.3582 - val_loss: 2.2170 - val_accuracy: 0.3557
Epoch 5/10
214/214 [==============================] - 16s 77ms/step - loss: 1.9021 - accuracy: 0.4253 - val_loss: 2.1590 - val_accuracy: 0.3886
Epoch 6/10
214/214 [==============================] - 17s 82ms/step - loss: 1.7554 - accuracy: 0.4759 - val_loss: 2.0464 - val_accuracy: 0.4235
Epoch 7/10
214/214 [==============================] - 17s 78ms/step - loss: 1.5497 - accuracy: 0.5342 - val_loss: 2.0235 - val_accuracy:

In [ ]:
results.to_pickle('results\\model_lstm')

array([[64, 0.2, 2, 32, 3.100442886352539,
        list([6.995808124542236, 3.100442886352539]),
        0.16634966433048248, 0.16634966433048248, 2.812389373779297,
        3.762976884841919, 0.18740805983543396, 0.18740805983543396]],
      dtype=object)

In [80]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])